In [1]:
pip install pandas fake_useragent

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached fake_useragent-1.5.1-py3-none-any.whl.metadata (15 kB)
  Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached fake_useragent-1.5.1-py3-none-any.whl (17 kB)
Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import concurrent.futures
import pandas as pd

In [ ]:
product_brands = []
product_names = []
current_prices = []
old_prices = []
ratings = []
product_urls = []

your_url = input(str('Please paste macy link with id=number at the end like https://www.macys.com/shop/handbags-accessories?id=26846'))

def scrape_macy(url):
    ua = UserAgent()
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    sales_box = soup.find_all('div', class_='product-thumbnail-container')


    for box in sales_box:
        #product_brand
        if box.find('div', class_ = 'product-brand medium') is not None:
            product_brand = box.find('div', class_ = 'product-brand medium').text.lstrip()
            product_brands.append(product_brand)
        else:
            product_brands.append('None')

                #product_name
        if box.find('div', class_ = "product-name medium") is not None:
            product_name = box.find('div', class_ = "product-name medium").text.lstrip() 
            product_names.append(product_name)
        else:
            product_names.append('None')  


            #price
        if box.find('span', class_ = "discount price-reg") is not None:
            current_price = box.find('span', class_ = "discount price-reg").text.strip() 
            current_prices.append(current_price)
        elif box.find('span', class_ = "price-reg") is not None:
            current_price = box.find('span', class_ = "price-reg").text.strip()
            current_prices.append(current_price)
        else:
            current_prices.append('None')  

                #old price 
        if box.find('span', class_ = "price-strike") is not None:
            old_price = box.find('span', class_ = "price-strike").text
            old_prices.append(old_price)
        else:
            old_prices.append('None')

            #rating
        if box.select_one('div.rating > div > fieldset') is not None:
            rating = box.select_one('div.rating > div > fieldset')
            rating = rating['aria-label']
            ratings.append(rating)
        else:
            ratings.append('None')


        if box.find('a', attrs={"href": True}) is not None:
            product_url = box.find('a', attrs={"href": True}) 
            product_url = product_url['href']
            product_urls.append('https://www.macys.com'+product_url)
        else:
            product_urls.append('None')

    if soup.find('a', class_ = 'pagination-next') is None:
        a = 1
        b = 'e'
        a+b
    else:
        pass


base_url, query_string = your_url.split('?', 1)
index = 1
while True:
    try: 
        page = '/Pageindex/'+ '{}?{}'.format(index,query_string)
        index += 1
        new_url = base_url+page
        scrape_macy(new_url)
        print(new_url)
        print(len(product_brands))
    except Exception as e:
        print(e)
        break


data = pd.DataFrame({
    'Product Brand': product_brands,
    'Product Name': product_names,
    'Current Price': current_prices,
    'Old Price': old_prices,
    'Rating': ratings,
    'Product URL': product_urls
})


product_description = []
link = []

def scrape_inner_page(url):

    ua = UserAgent()
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    new_page = requests.get(url, headers = headers)
    new_soup = BeautifulSoup(new_page.content, "html.parser")

    if new_soup.find('li', attrs={'data-auto': 'product-summary-section'}) is not None:
        details = new_soup.find('li', attrs={'data-auto': 'product-summary-section'})
        product_description.append(details.text)
    else:
        product_description.append('None')

    link.append(url)
    print(url)


with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    executor.map(scrape_inner_page,product_urls)


data_inner = pd.DataFrame({
    'Product Description': product_description,
    'Product URL': link
})

result = pd.merge(data, data_inner, left_on='Product URL', right_on='Product URL', how='inner')
result.to_csv(f'{base_url.split('/')[-1]}.csv')
